In [87]:
import datetime
import requests
import hashlib
import json
import pandas as pd
from math import ceil

In [88]:
#Define constants
timestamp = datetime.datetime.now().isoformat()
public_key = 'put your plublic key here'
private_key = 'put your private key here'

input_hash = timestamp + private_key + public_key

In [89]:
#Define hash function
def hash_md5(x):
    md5 = hashlib.md5()
    md5.update(x)
    result = md5.hexdigest()
    
    return result

In [90]:
#Define get_data() function to get data from API and return a dictionary
def get_data(limit,off):
    parameters= {'ts':timestamp, 'apikey': public_key, 'hash': hash_md5(input_hash.encode('utf-8')), 'limit':limit, 'offset':off}
    response = requests.get('https://gateway.marvel.com:443/v1/public/characters', params=parameters)
    data_json = json.loads(response.text)
    
    return data_json

In [91]:
#Define the number of API calls
iterations = ceil(get_data(1,0)['data']['total']/100)

In [92]:
#Create a list for each column
Charecters_List =  ['id','name','description','comics','series','stories','events']
d = {}  
for i in Charecters_List: 
    d[i] = []

In [93]:
#Extract and append data to the list
offset=0
for i in range(iterations):
    
    data_json = get_data(100,offset)
        
    for element in data_json['data']['results']:
        d['id'].append(element['id'])
        d['name'].append(element['name'])
        d['description'].append(element['description'])
        d['comics'].append(element['comics']['available'])
        d['series'].append(element['series']['available'])
        d['stories'].append(element['stories']['available'])
        d['events'].append(element['events']['available'])
    
    offset=offset+100

In [94]:
#Transform in a pandas DataFrame
df = pd.DataFrame.from_dict(d, orient='index')
df = df.transpose()

In [95]:
df

,id,name,description,comics,series,stories,events
0,1011334,3-D Man,,12,3,21,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,3,2,7,0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,49,33,52,0
3,1010699,Aaron Stack,,14,3,27,0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",53,26,63,1
...,...,...,...,...,...,...,...
1488,1010780,Zemo,,15,9,12,1
1489,1011127,Zodiak,"Twelve demons merged with Norman Harrison, who...",1,1,1,0
1490,1009741,Zombie (Simon Garth),War hero Simon Garth was turned into a zombie ...,1,1,1,0
1491,1011183,Zuras,Zuras was once the leader of the Eternals.,3,3,3,0
